In [1]:
import json
import pandas as pd

from collections import defaultdict, Counter
from abandabot.evaluate import evaluate_performance

In [2]:
with open("reports/performance.json") as f:
    perf = json.load(f)

In [3]:
table = []
for setting, perf_logs in perf.items():
    model = setting.split("+")[0]
    context = "+".join(setting.split("+")[1:])
    avg_perf = pd.DataFrame(perf_logs).mean()
    std_pref = pd.DataFrame(perf_logs).std()
    results = {"model": model, "context": context}
    for key in avg_perf.keys():
        results[key] = f"{avg_perf[key]:.3f}±{std_pref[key]:.3f}"
    table.append(results)

ground_truth = pd.read_csv("ground_truth.csv")
ground_truth = ground_truth[ground_truth.impactful.isin(("Yes", "No"))]

table.append(
    {
        "model": "yesman",
        "context": "yesman",
        **evaluate_performance(
            ground_truth.impactful, ["Yes"] * len(ground_truth), "Yes", "No"
        ),
    }
)

table.append(
    {
        "model": "noman",
        "context": "noman",
        **evaluate_performance(
            ground_truth.impactful, ["No"] * len(ground_truth), "Yes", "No"
        ),
    }
)

table.append(
    {
        "model": "random",
        "context": "random",
        "errors": 0,
        "macro_precision": 0.5,
        "macro_recall": 0.5,
        "macro_f1": 0.5,
    }
)

table = pd.DataFrame(table)
table["model context macro_precision macro_recall macro_f1".split()]

,model,context,macro_precision,macro_recall,macro_f1
0,gpt-4o,no+context+no+reasoning,0.737±0.061,0.638±0.043,0.648±0.053
1,gpt-4o,no+context,0.739±0.039,0.618±0.027,0.623±0.036
2,gpt-4o,no+reasoning,0.685±0.026,0.687±0.026,0.686±0.026
3,gpt-4o,context+reasoning,0.672±0.037,0.657±0.039,0.661±0.039
4,gpt-4o,context+reasoning+complex,0.743±0.047,0.752±0.045,0.746±0.046
5,deepseek-v3,no+context+no+reasoning,0.780±0.035,0.633±0.016,0.643±0.021
6,deepseek-v3,no+context,0.753±0.049,0.581±0.025,0.566±0.038
7,deepseek-v3,no+reasoning,0.665±0.009,0.694±0.010,0.643±0.010
8,deepseek-v3,context+reasoning,0.641±0.032,0.655±0.036,0.643±0.034
9,deepseek-v3,context+reasoning+complex,0.740±0.026,0.772±0.029,0.746±0.028


In [4]:
table[table.context == "context+reasoning+complex"]

,model,context,errors,yes_precision,yes_recall,yes_f1,no_precision,no_recall,no_f1,macro_precision,macro_recall,macro_f1
4,gpt-4o,context+reasoning+complex,1.200±1.398,0.857±0.029,0.825±0.045,0.840±0.031,0.628±0.072,0.679±0.072,0.651±0.062,0.743±0.047,0.752±0.045,0.746±0.046
9,deepseek-v3,context+reasoning+complex,0.200±0.422,0.889±0.022,0.761±0.029,0.820±0.022,0.590±0.036,0.783±0.047,0.672±0.036,0.740±0.026,0.772±0.029,0.746±0.028
14,llama-v3p3,context+reasoning+complex,0.000±0.000,0.789±0.018,0.912±0.022,0.846±0.018,0.690±0.064,0.444±0.051,0.540±0.054,0.740±0.040,0.678±0.032,0.693±0.035
19,gemini-2.0,context+reasoning+complex,0.000±0.000,0.755±0.013,0.977±0.020,0.852±0.013,0.854±0.122,0.276±0.048,0.414±0.060,0.805±0.064,0.627±0.027,0.633±0.035
